<a href="https://colab.research.google.com/github/ajkc-dev/great_courses_ml/blob/master/build_spam_ditector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **In this noteobook**, we will build a spam detector using two different models, a decision tree and a naive Bayes model. A naive Bayes classifier calculates the probability of a sequence of words belonging to a class as proprotional to the product of the probability of each item in a sequence given the class.

Below we import the libraries we'll be using.


In [ ]:
from sklearn import tree
import graphviz
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

Next we'll write a function to process the data into a dictionary of words and their number of occurances, `word_dict`, and a count of the number of words total, `lexiconsize`

In [ ]:
# read in the vocabulary file
def readvocab(vocab_path="vocab.txt"):
   # keep track of the number of words
    lexiconsize = 0
   # initialize an empty dictionary
    word_dict = {}
   # create a feature for unknown words
    word_dict["@unk"] = lexiconsize
    lexiconsize += 1
   # read in the vocabular file
    with open(vocab_path, "r") as f:
        data = f.readlines()
   # Process the file a line at a time.
    for line in data:
        # The count is the first 3 characters
        count = int(line[0:4])
        # The word is the rest of the string
        token = line[5:-1]
       # Create a feature if it’s appeared at least twice
        if count > 1:
            word_dict[token] = lexiconsize
            lexiconsize += 1
    # squirrel away the total size for later reference
    word_dict["@size"] = lexiconsize
    return(word_dict)

We will download the vocabulary data from GitHub, `vocab.txt`


In [ ]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/vocab.txt

Next, we write a `tokenize` function to turn each word into a list of the length of the number words.  Every item in the list is a count of the number of times a given word occurs in the list.

In [ ]:
# Turn string str into a vector.
def tokenize(email_string, word_dict):
  # initially the vector is all zeros
  vec = [0 for i in range(word_dict["@size"])]
  # for each word
  for t in email_string.split(" "):
   # if the word has a feature, add one to the corresponding feature
    if t in word_dict: vec[word_dict[t]] += 1
   # otherwise, count it as an unk
    else: vec[word_dict["@unk"]] += 1
  return(vec)

From here, we write a `getdat` function to convert the file we downloaded into two lists:

- `dat`: a list of lists of tokenized words
- `labs`: a list of labels associated with the email being spam or not spam

In [ ]:
# read in labeled examples and turn the strings into vectors
def getdat(filename, word_dict):
    with open(filename, "r") as f:
        data = f.readlines()
    dat = []
    labs = []
    for line in data:
        labs = labs + [int(line[0])]
        dat = dat + [tokenize(line[2:], word_dict)]
    return(dat, labs)

Now we'll download the train and test data from GitHub

In [ ]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/spam-test.csv
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/spam-train.csv

With these train and test datasets, we'll build create the data and labels we will use to train and use to test our naive Bayes model.

In [ ]:
word_dict = readvocab()
traindat, trainlabs = getdat("spam-train.csv", word_dict)
testdat, testlabs = getdat("spam-test.csv", word_dict)

With the training and testing data, we can fit a decision tree with 6 decision rules and print out the accuracy on the test data

In [ ]:
clf = tree.DecisionTreeClassifier(max_leaf_nodes = 6)
clf = clf.fit(traindat, trainlabs)

yhat = clf.predict(testdat)

sum([yhat[i] == testlabs[i] for i in range(len(testdat))])/len(testdat)

We now will create a list of the words in our wordlist and use it to print the decision tree we have learned

In [ ]:
wordlist = list(word_dict.keys())[:-1]
dot_data = tree.export_graphviz(clf, feature_names=wordlist,
                      filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph

How does the number of decision rules affect the accuracy of the model? We'll retrain the model 29 times to see how the accuracy changes as we increase the number of decision rules from 2 to 30.

In [ ]:
for leaves in range(2, 31):
  clf = tree.DecisionTreeClassifier(max_leaf_nodes = leaves)
  clf = clf.fit(traindat, trainlabs)
  yhat = clf.predict(testdat)
  acc = sum([yhat[i] == testlabs[i] for i in range(len(testdat))])/len(testdat)
  print(leaves,acc)

Let's now fit a naive Bayes model and print the accuracy of the model


In [ ]:
clf = MultinomialNB().fit(traindat, trainlabs)
clf = clf.fit(traindat, trainlabs)
yhat = clf.predict(testdat)
acc = sum([yhat[i] == testlabs[i] for i in range(len(testdat))])/len(testdat)
acc

We can also calculate the confusion matrix of the model, a table of the following counts:

True Negatives False Positives

False Negatives True Positives

In [ ]:
print(confusion_matrix(testlabs, yhat))

Let's visualize how Naive Bayes combines information from words in a sentence to make a judgement.

In [ ]:
import numpy as np

def plotsentence(sentence, clf):
  acc = 1.0
  labs = []
  facs = []
  factor = np.exp(clf.class_log_prior_[0]- clf.class_log_prior_[1])
  labs += ["PRIOR"]
  facs += [factor]
  acc *= factor
  for w in sentence:
    i = word_dict[w]
    factor = np.exp(clf.feature_log_prob_[0][i]- clf.feature_log_prob_[1][i])
    labs += [w]
    facs += [factor]
    acc *= factor
  labs += ["POST"]
  facs += [acc]
  return((labs,facs))

In [ ]:
(labs,facs) = plotsentence(['yo', 'come', 'over', 'carlos', 'will', 'be', 'here', 'soon'], clf)
facs = [ fac if fac >= 1.0 else -1/fac for fac in facs ]
[(l,round(f,1)) for (l,f) in zip(labs,facs)]

In [ ]:
(labs,facs) = plotsentence(['congratulations', 'thanks', 'to', 'a', 'good', 'friend', 'u', 'have', 'won'], clf)
facs = [ fac if fac >= 1.0 else -1/fac for fac in facs ]
[(l,round(f,1)) for (l,f) in zip(labs,facs)]